# Density RF Model 

Barebones notebook to reproduce the Murphy et al. [2024] density model. 

Only the model is reproduced, no testing is done (residuals, hyperparameters, permutation importance, etc.). This was done in another set of analysis and removing it here simplifies the notebook. 

In [1]:
import pandas as pd
import numpy as np
import time
import gc
import pickle
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [2]:
# random state and random forest parameters
# random state ensures the same model is generated

rnd=17
rf_params = {
    "n_estimators": 500,
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf":5,
    "warm_start":False,
    "oob_score":True,
    "random_state": rnd,
    "max_features":0.5,
    "n_jobs":10
    }

In [3]:
def dat_create(dat, col, log_col, lt_col, y_col, t_col):

    x_dat = dat[col+t_col+[y_col]].dropna().copy()

    if log_col:
       for i in log_col:
            try:
                x_dat[i] = np.log10(x_dat[i])
            except:
                print(f'Could not log column {i}')
    
    if lt_col:
        for i in lt_col:
            try:
                if dat[i].max() > 24:
                    x_dat[f'cos_{i}'] = np.cos(dat[i]*2*np.pi/360.)
                    x_dat[f'sin_{i}'] = np.sin(dat[i]*2*np.pi/360.)
                else:
                    x_dat[f'cos_{i}'] = np.cos(dat[i]*2*np.pi/24.)
                    x_dat[f'sin_{i}'] = np.sin(dat[i]*2*np.pi/24.)    
            except:
                print(f'Could not add {i} as a cos/sin time column')
    
    x_dat = x_dat[~x_dat.isin([np.nan, np.inf, -np.inf]).any(axis=1)].dropna()
    y_dat = x_dat[y_col].copy()
    x_dat = x_dat.drop(columns=y_col)    
    
    return x_dat, y_dat

In [7]:
def rf_model(col=['1300_02', 'SYM_H index','SatLat'], 
             y_col='400kmDensity', 
             t_col=['DateTime'], 
             log_col=['1300_02'], 
             lt_col=['SatMagLT'], 
             rf_params=rf_params, 
             target_dat='D:\\data\\SatDensities\\satdrag_database_grace_B.hdf5', 
             oos_dat='D:\\data\\SatDensities\\satdrag_database_grace_A.hdf5',
             oos_dat2='D:\\data\\SatDensities\\satdrag_database_grace_CHAMP_SI_int.hdf5',
             n_repeats=10):
    
    
    rnd = rf_params['random_state']

    dat_dic = {'feature_cols':col,
               'target_cols':y_col,
               'time_cols':t_col,
               'log_col':log_col,
               'lt_col':lt_col}
    
    kcol = [col,[y_col],t_col,lt_col]
    kflt = [item for sublist in kcol for item in sublist]
    df = pd.read_hdf(target_dat)
    df = df[kflt].dropna()

    reg_x, reg_y = dat_create(dat=df,col=col,log_col=log_col,lt_col=lt_col,
                              y_col=y_col,t_col=t_col)
    reg_y = reg_y*(10**12)
    

    # create data set from out of sample data
    df_oos = pd.read_hdf(oos_dat)
    oos_x, oos_y = dat_create(dat=df_oos,col=col,log_col=log_col,lt_col=lt_col,
                              y_col=y_col,t_col=t_col)
    oos_y = oos_y*(10**12)
    oos_t = oos_x[t_col]
    oos_x = oos_x.drop(columns=t_col)
    

    df_oos2 = pd.read_hdf(oos_dat2)
    oos_x2, oos_y2 = dat_create(dat=df_oos2,col=col,log_col=log_col,lt_col=lt_col,
                                y_col=y_col,t_col=t_col)
    oos_y2 = oos_y2*(10**12)
    oos_t2 = oos_x2[t_col]
    oos_x2 = oos_x2.drop(columns=t_col)

    del df
    del df_oos
    del df_oos2
    gc.collect
    
    # create train test splits
    train_x, test_x, train_y, test_y = train_test_split(reg_x, reg_y, 
                                                        test_size=0.3, 
                                                        random_state=rnd)

    # get and drop DateTime column
    train_t = train_x[t_col].copy()
    test_t = test_x[t_col].copy()

    train_x = train_x.drop(columns=t_col)
    test_x = test_x.drop(columns=t_col)

    print('Train and fit model')

    start = time.time()
    print("Time elapsed working on RandomForest")

    rfr = RandomForestRegressor(**rf_params)
    rfr.fit(train_x, train_y)

    end = time.time()
    print("Time consumed in working: ",end - start)

    #Make predictions and calculate error
    predictions = rfr.predict(test_x)
    pre_oos = rfr.predict(oos_x)
    pre_oos2 = rfr.predict(oos_x2)
    pre_tr = rfr.predict(train_x)
    
    # combine data sets into single dataframes
    train_d = train_x.join([train_y,train_t], how='left')
    test_d = test_x.join([test_y,test_t], how='left')
    oos_d = oos_x.join([oos_y,oos_t], how='left')
    oos2_d = oos_x2.join([oos_y2,oos_t2], how='left')
    
    # add predictions to the dataframes
    train_d[y_col+'_pred'] = pre_tr
    test_d[y_col+'_pred'] = predictions
    oos_d[y_col+'_pred'] = pre_oos
    oos2_d[y_col+'_pred'] = pre_oos2
    
    
    
    return rfr, train_d, test_d, oos_d, oos2_d, dat_dic

In [8]:
def rf_run(y_col='400kmDensity', 
           lt_col=['SatMagLT'],
           pre_f = False,
           app_f = False
           ):
    """
    Run a set of random forest models 

    Returns
    -------
    None.
    
    Saves data frames to file for subsequent analysis

    """
    
    # out_dir 
    o_dir = 'D:\\data\\SatDensities\\'
    
    # repeats for permutation importance
    n_repeats = 5
    # columns that are not used in the model but are returned
    # to make subsequent analysis easier
    t_col = ['DateTime','storm','storm phase']
    
    # columns to log for fism and geo datasets
    fi_log = ['1300_02', '43000_09', '85550_13', '94400_18']
    
    # solar indice columns
    si_col = ['F10', 'F81', 'S10', 'S81c', 'M10', 'M81c', 'Y10', 'Y81c', 'SatLat']
            
    # fism2 columns
    fi_col = ['1300_02', '43000_09', '85550_13', '94400_18', 'SatLat']
 
    # fism2 and geo columns
    fgeo_col = ['1300_02', '43000_09', '85550_13', '94400_18', 'SYM_H index', 'AE', 'SatLat']

    # labels
    data_labels = ['SI','FI','FI_GEO']
    data_sets = [si_col, fi_col, fgeo_col]

    data_labels = ['FI_GEO']
    data_sets = [fgeo_col]
    
    for col, d_in in zip(data_sets,data_labels):
        
        print(d_in)

        rf_dat = rf_model(y_col=y_col, lt_col=lt_col,
                          col=col, t_col=t_col, log_col=fi_log, 
                          n_repeats=n_repeats)
        
        fn = f'{d_in}_RFdat'
        if pre_f:
            fn = f'{pre_f}{fn}'
        if app_f:
            fn = f'{fn}{app_f}'
            
        fn = f'{fn}.pkl'
        fn = os.path.join(o_dir,fn)
        
        with open(fn, 'wb') as f:
            pickle.dump(rf_dat, f)
            
        del rf_dat
        gc.collect

In [9]:
rf_run(app_f='_AIMFAHR')

FI_GEO


c:\Users\murph\miniconda3\envs\satdrag\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\murph\miniconda3\envs\satdrag\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\murph\miniconda3\envs\satdrag\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Train and fit model
Time elapsed working on RandomForest
Time consumed in working:  222.15123224258423
